In [1]:
import json
import requests
from datetime import datetime
import pytz

In [2]:
currency_pair = "btcusd"
# ohlc_url = f"https://www.bitstamp.net/api/v2/ticker/{currency_pair}/"
ohlc_url = f'https://www.bitstamp.net/api/v2/ohlc/{currency_pair}/?step=3600&limit=1'
hour_url = "http://127.0.0.1:8000/scrap/api/bitstamphour/add"
day_url = "http://127.0.0.1:8000/scrap/api/bitstamp/add"
error_log_url = "http://127.0.0.1:8000/scrap/api/error_log/add/"

In [4]:
def call_bitsamp_api(ohlc_url,api_endpoint,error_log_url):
    ohlc_data_dict = {}

    try:
        data = requests.get(ohlc_url)

        bitstamp_data = data.json()['data']

        ohlc_data = bitstamp_data['ohlc'][0]
        ohlc_data['unix'] = ohlc_data.pop('timestamp')
        ohlc_data['volume_btc'] = ohlc_data.pop('volume')
        # Convert the Unix timestamp to UTC time
        ohlc_data['date'] = datetime.utcfromtimestamp(int(ohlc_data['unix'])).strftime('%Y-%m-%dT%H:%M:%S')
        
        # PROD
        # ohlc_data['symbol'] = bitstamp_data['pair']

        # TEST
        ohlc_data['symbol'] = 'TEST'

        # Convert the timestamp to datetime and localize it to UTC
        dt = datetime.fromtimestamp(int(ohlc_data['unix']), tz=pytz.UTC)
        # Convert the datetime to Kuala Lumpur time
        kl_time = dt.astimezone(pytz.timezone('Asia/Kuala_Lumpur'))
        ohlc_data['local_date'] = kl_time.strftime('%Y-%m-%dT%H:%M:%S')

        # Convert 'volume' and 'close' to float before multiplication
        ohlc_data['volume_usd'] = float(ohlc_data['volume_btc']) * float(ohlc_data['close'])
        
        ohlc_json = json.dumps(ohlc_data)

        # Convert the string to a dictionary
        ohlc_data_dict = json.loads(ohlc_json)

    except Exception as e:
        print(e)
        error = {
            "error_message": e.message if hasattr(e, 'message') else str(e),
            "error_source": 'when calling bitstamp official api',
            "error_type": 'bitstamp api error',
            "error_content": data.to_dict() if hasattr(data, 'to_dict') else str(data)
        }
        requests.post(error_log_url, data=error)
        
    try:
        response = requests.post(api_endpoint, json=ohlc_data_dict)
        if response.status_code == 200:
            print('Data successfully added to the database')
        else:

            error_data = {
                "error_source": 'Bitstamp API',
                "error_type": response.text,
                "error_content": api_endpoint,
                "error_message": "Calling bitstamp store api"
            }
            requests.post(error_log_url, json=error_data)
    except requests.exceptions.RequestException as e:
        error_data = {
        "error_source": 'Bitstamp API',
        "error_type": response.text,
        "error_content": api_endpoint,
        "error_message": e.message if hasattr(e, 'message') else str(e),
        }
        requests.post(error_log_url, json=error_data)
        print(e)


In [5]:
call_bitsamp_api(ohlc_url,hour_url,error_log_url)
call_bitsamp_api(ohlc_url,day_url,error_log_url)

C:\Users\Zhong_ming_Tan\AppData\Local\Temp\ipykernel_24932\3175002737.py:11: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  ohlc_data['date'] = datetime.datetime.utcfromtimestamp(int(ohlc_data['unix'])).strftime('%Y-%m-%dT%H:%M:%S')


Data successfully added to the database
Data successfully added to the database
